In [36]:
import pandas as pd
import time
import multiprocessing as mp
from multiprocessing import Pool
import numpy as np

In [4]:
#import the dataset
dataframe = pd.read_csv('/Users/lindseyclark/Documents/formula_1_project/formula-1-race-data-19502017/lapTimes.csv')

In [5]:
#describe the dataset
dataframe.describe()

,raceId,driverId,lap,position,milliseconds
count,426633.000000,426633.000000,426633.000000,426633.000000,4.266330e+05
mean,423.120853,186.503817,29.827172,9.646896,9.580222e+04
std,381.542422,316.123374,18.402245,5.580664,7.475541e+04
min,1.000000,1.000000,1.000000,1.000000,6.741100e+04
25%,100.000000,14.000000,14.000000,5.000000,8.238200e+04
50%,205.000000,26.000000,29.000000,9.000000,9.080000e+04
75%,881.000000,71.000000,44.000000,14.000000,1.027380e+05
max,988.000000,843.000000,78.000000,24.000000,7.507547e+06


In [14]:
#the dataset has a 'time' columnt that we will split
dataframe

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
426628,988,825,50,13,1:43.928,103928
426629,988,825,51,13,1:44.138,104138
426630,988,825,52,13,1:43.934,103934
426631,988,825,53,13,1:44.164,104164


In [23]:
def create_split_cols(dataframe):
    dataframe2 = dataframe.time.apply(lambda x: pd.Series(str(x).split(".")))
    return dataframe2

In [25]:
#without multiprocessing
t0 = time.time()
dataframe2 = create_split_cols(dataframe)
t1 = time.time()
time_elapsed = t1-t0
print(dataframe2)
print(time_elapsed)

           0    1
0       1:38  109
1       1:33  006
2       1:32  713
3       1:32  803
4       1:32  342
...      ...  ...
426628  1:43  928
426629  1:44  138
426630  1:43  934
426631  1:44  164
426632  1:44  285

[426633 rows x 2 columns]
80.83246326446533


In [28]:
#with multiprocessing
p = mp.Pool(mp.cpu_count()) # Data parallelism Object
t2 = time.time()
dataframe3 = p.map(create_split_cols(dataframe), dataframe)
#b = p.map(clean_text, df2['Plot'])
t3 = time.time()
time_elapsed2 = t3-t2
print(dataframe2)
print(time_elapsed2)
#print("time consuming after Parallel Processing to process the Dataset {0:.2f}s".format(round(t3-t2, 2)))

Process ForkPoolWorker-21:
Process ForkPoolWorker-17:
Process ForkPoolWorker-23:
Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-24:
Process ForkPoolWorker-18:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Li

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [37]:
num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [39]:
t2 = time.time()
parallelize_dataframe(dataframe, create_split_cols)
t3 = time.time()
time_elapsed2 = t3-t2
print(time_elapsed2)

27.803346872329712
